In [104]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: ANYWIDGET_HMR=1


In [105]:
import chromospyce
import numpy as np

In [110]:
import random

def random_walk_3D(n_steps):
    position = np.array([0.0, 0.0, 0.0])
    positions = [position.copy()]
    for _ in range(n_steps):
        step = np.random.choice([-1.0, 0.0, 1.0], size=3)  # Randomly choose to move left, right, up, down, forward, or backward
        position += step
        positions.append(position.copy())
    return positions


In [113]:
## Use case 1: Passing raw numpy array
test_data = make_random_3D_chromatin_structure(n=100)
# data = np.array(test_data)
data = np.array(random_walk_3D(50))
# print(data)

chromospyce.Widget(structure_nparray=data.tobytes())

Widget(structure_nparray=b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0…

In [114]:
## Use case 2: Passing XYZ string
test_string = """
6
sample = 0
CA	1.101124	0.547027	-2.299305
CA	-0.919687	1.275041	-3.201912
CA	-0.974526	1.144577	-4.491504
CA	-0.881394	-0.011930	-4.496625
CA	-0.455539	-2.518781	-2.500660
CA	-0.626368	-2.086400	-3.427676
"""

chromospyce.Widget(structure_string=test_string, delimiter="\t")

Widget(delimiter='\t', structure_string='\n6\nsample = 0\nCA\t1.101124\t0.547027\t-2.299305\nCA\t-0.919687\t1.…

In [115]:
## Use case 3: Passing XYZ file path
import pathlib
test_path = pathlib.Path("./sample_data/test.xyz")

chromospyce.Widget(structure_path=test_path, delimiter="\t")

Widget(delimiter='\t', structure_path=PosixPath('sample_data/test.xyz'))